In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#First Dataset containing details of groceries
df_1 = pd.read_csv("/kaggle/input/groceries-dataset-for-market-basket-analysismba/Groceries data.csv")
df_1.head(20)

In [ ]:
#Second Dataset containing information about items ordered
df_2 = pd.read_csv("/kaggle/input/groceries-dataset-for-market-basket-analysismba/basket.csv")
df_2.head(10)

In [ ]:
 #To check how many customers ordered all 11 items
df_2[df_2.notna().all(axis=1)].head(10)

In [ ]:
#Concatenating the datasets together
df = pd.concat([df_1,df_2], axis =1)

***Analysing the Dataset***

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#Importing necessary Libraries for Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

***Looking for insights from the Dataset***

In [ ]:
customer_details = df.groupby(["Member_number",'year'])["year"].value_counts().reset_index()
customer_details

In [ ]:
top_10_customers = customer_details.sort_values(by = 'count', ascending = False).head(10)
top_10_customers

In [ ]:
top_10_customers = top_10_customers.query('count>22')
top_10_customers

In [ ]:
plt.figure(figsize= (8,8))
sns.barplot(top_10_customers, x = "Member_number",y = "count", palette = 'viridis')
plt.xlabel("Members ID")
plt.ylabel("Number of yearly orders")
plt.title("Top 10 Customers")
plt.show()

In [ ]:
most_ordered_month = df.groupby(["month"])["month"].value_counts().reset_index()
most_ordered_month

In [ ]:
most_ordered_items = df.groupby(["itemDescription"])["itemDescription"].value_counts().reset_index()
most_ordered_items

In [ ]:
top_10_items = most_ordered_items.sort_values(by= 'count', ascending = False).head(10)
top_10_items

In [ ]:
top_10_items = top_10_items.query('count>811')
top_10_items

In [ ]:
plt.figure(figsize=(8,8))
sns.barplot(top_10_items,x="itemDescription", y = "count", palette = "deep")
plt.xlabel("Item Description")
plt.xticks(rotation = 90)
plt.ylabel("Number of Orders")
plt.title("Top 10 Items Ordered")
plt.show()

In [ ]:
#!pip install mlxtend

***Performing Market Basket Analysis***

In [ ]:
#Importing necessary libraries
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
df_2.fillna('1', inplace = True)

Transactions=[]

for i in range (14963):
    transaction = []
    for j in range(11):
        if df_2.iloc[i,j] != '1':
            transaction.append(df_2.iloc[i,j])
    Transactions.append(transaction)
            

In [ ]:
te = TransactionEncoder()
te_bin = te.fit_transform(Transactions)
Transactions = pd.DataFrame(te_bin, columns = te.columns_)

In [ ]:
Transactions

In [ ]:
def encode(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
Transactions = Transactions.applymap(encode)

In [ ]:
Transactions

In [ ]:
Frequent_itemset = apriori(Transactions, min_support = 0.002, use_colnames = True)
Frequent_itemset.head()

In [ ]:
rules = association_rules(Frequent_itemset, metric = "lift", min_threshold = 1) 
rules

In [ ]:
sorted_rules = rules.sort_values(by ="lift", ascending = False)
sorted_rules.head(10)